In [1]:
# source ~/mx_venv/bin/activate
# cd /mk_utils/MXNet2Caffe
# python -m json2prototxt --mx-json /home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/train_7-symbol.json --cf-prototxt /home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/model-caffe-converted.prototxt
# remove all layers after 'out_embedding_scale'
# save modified file as dgx_train7_58-caffe.prototxt
# python -m mxnet2caffe --mx-model /home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/train_7 --mx-epoch 58 --cf-prototxt /home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/dgx_train7_58-caffe.prototxt --cf-model /home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/dgx_train7_58-caffe-converted.caffemodel

'\nlayer {\n  bottom: "out_relu3"\n  top: "flatten"\n  name: "fc1_flatten"\n  type: "Flatten"\n  flatten_param { axis: 1}\n}\n'

In [1]:
import sys
sys.path.append('/home/macul/Documents/macul/MXNet2Caffe')
sys.path.append('/home/macul/Documents/macul1/insightface/src/')
sys.path.append('/home/macul/Documents/macul/mklib/nn/mxiter')
sys.path.append('/home/macul/Documents/macul/mklib/nn/mxconv')
import mxnet as mx
import caffe
from collections import namedtuple
import numpy as np
import cv2

In [2]:
#mxModelPath = '/media/macul/black/mxnet_training/r50/dgx_train1/train_1'
#mxModelEpoch = 31
mxModelPath = '/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/train_7'
mxModelEpoch = 58

sym_tmp, arg_params, aux_params = mx.model.load_checkpoint(mxModelPath, mxModelEpoch)

In [3]:
all_layers = sym_tmp.get_internals()
output_list = all_layers.list_outputs()
sym3 = all_layers['out_embedding_output']
#sym3 = all_layers['out_fc1_output']
model = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
model.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])
model.set_params(arg_params, aux_params, allow_missing=False, allow_extra=True)

In [5]:
output_list

['data',
 'stem_conv1_weight',
 'stem_conv1_output',
 'stem_bn2_gamma',
 'stem_bn2_beta',
 'stem_bn2_moving_mean',
 'stem_bn2_moving_var',
 'stem_bn2_output',
 'stem_relu1_gamma',
 'stem_relu1_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_prelu2_gamma',
 'stage1_unit1_prelu2_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn4_gamma',
 'stage1_unit1_bn4_beta',
 'stage1_unit1_bn4_moving_mean',
 'stage1_unit1_bn4_moving_var',
 'stage1_unit1_bn4_output',
 'stage1_unit1_convr_weight',
 'stage1_unit1_convr_output',
 'stage1_unit1_bnr_gamma',
 'stage1_unit1_bnr_beta',
 'stage1_unit1_bnr_moving_mean',
 'stage1_unit1_bnr_moving_var',
 

In [4]:
img_normalization = True
img = mx.image.imread('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0001.png')
img = img.astype('float32')
if img_normalization:
    img = img - 127.5
    img = img * 0.0078125
img = img.transpose((2,0,1))
img = img.expand_dims(axis=0)   
Batch = namedtuple('Batch',['data'])
model.forward(Batch([img]))

In [5]:
outs=model.get_outputs()

In [6]:
out_mx=outs[0].asnumpy()[0]

In [7]:
print(arg_params['out_fc1_bias'].shape)
print(arg_params['out_fc1_weight'].shape)

(512L,)
(512L, 25088L)


In [9]:
arg_params['out_embedding_gamma']=1

In [2]:
cfModelProto='/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/resnet20_no_merge_no_flattern.prototxt'
cfModelWeight='/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/resnet20_no_merge_no_flattern.caffemodel'
#cfModelProto='/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/dgx_train7_58-caffe.prototxt'
#cfModelWeight='/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/dgx_train7_58-caffe-converted.caffemodel'
#cfModelProto='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.prototxt'
#cfModelWeight='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe-converted.caffemodel'
Net = caffe.Net(cfModelProto, cfModelWeight, caffe.TEST)
Net.blobs['data'].reshape(1,3,112,112)

In [9]:
def getOutput(img_path, output_name):
    mean_array = np.ones([3,112, 112])*127.5
    scale_value = 0.0078125
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = np.rollaxis(img,2)
    img = img - mean_array
    img = img * scale_value
    img = img[np.newaxis,...] 
    
    Net.blobs['data'].data[...] = img
    out = Net.forward()
    out = out[output_name][0]
    return out

In [7]:
print(Net.params['stage1_unit1_bn1'][0].data[...])
print(Net.params['stage1_unit1_bn1'][1].data[...])
print(Net.params['stage1_unit1_bn1'][2].data[...])

[-5.60519386e-45  5.60519386e-45  1.10433064e-02  4.30759148e-42
 -5.60519386e-45  2.94192211e-34  5.60519386e-45  1.08831981e-03
  1.03809759e-02  1.10923755e-03  1.43937301e-03  4.35527181e-03
  9.63252564e-42  5.60519386e-45  1.75862957e-42  2.96234495e-42
  3.35522997e-03  1.24505369e-41  1.21942218e-02  3.93827306e-03
  5.60519386e-45  4.57037135e-26  1.03806658e-02  1.40949222e-03
  5.60519386e-45  5.60519386e-45  1.31598385e-02  5.60519386e-45
  1.24575542e-02  2.42204126e-02 -5.60519386e-45  5.78949824e-02
  5.60519386e-45  1.61340041e-03  5.60519386e-45  9.52042177e-42
  2.79208719e-41 -5.60519386e-45  1.03718266e-02  5.60519386e-45
 -5.60519386e-45  1.03112478e-02  3.13890856e-43  1.80787928e-02
  4.08338373e-42  1.62425954e-02  5.60519386e-45 -5.60519386e-45
 -5.60519386e-45  1.04078809e-02  5.60519386e-45  5.60519386e-45
  1.11450872e-40 -5.60519386e-45 -5.60519386e-45  5.60519386e-45
  2.00594263e-03  2.86134053e-03  5.60519386e-45  5.15004213e-04
  5.60519386e-45  3.24400

In [11]:
print(Net.params['stage1_unit1_bn1_scale'][0].data[...])
print(Net.params['stage1_unit1_bn1_scale'][1].data[...])

[ 4.05306779e-32  1.14878448e-41  3.74558344e-02  3.08846182e-42
 -8.62541244e-41 -1.05287560e-32  9.25978025e-42  3.84294949e-02
  4.99222614e-02  3.68555672e-02  4.32085171e-02  1.98072139e-02
 -3.57050849e-42 -9.63252564e-42 -1.39513275e-41  9.73902433e-43
  3.09604052e-02  1.08838852e-41  3.74050625e-02  2.54921988e-02
  1.23972875e-41  2.90541317e-25  4.46953438e-02  2.86253858e-02
  1.24399851e-16 -9.45245879e-41  3.93973701e-02  1.07171307e-41
  4.25768830e-02  9.08718817e-03  2.80423645e-40  1.49323829e-02
 -7.71835194e-42  3.21382917e-02  3.80732793e-42 -7.10178062e-42
 -7.60792962e-41  1.90844461e-21  6.43513575e-02 -4.42810315e-42
  2.58539567e-42  6.30221888e-02 -4.88072255e-42 -1.29156550e-02
 -2.72692681e-42  3.10110245e-02  1.30460887e-42 -8.02357699e-26
  1.09931865e-41  5.14237210e-02  6.03258989e-42  1.18409720e-41
  4.24859681e-41 -3.76158042e-30 -1.12840235e-26  6.49641968e-42
  1.88972093e-02  1.92036461e-02  4.80134439e-08  3.84735242e-02
  6.74304821e-42 -1.15845

In [10]:
print(Net.params['stage1_unit1_conv1'][0].data[...].shape)
print(Net.params['stage1_unit1_conv1'][1].data[...].shape)

(64, 64, 3, 3)


IndexError: Index out of range

In [10]:
Net.params['out_fc1'][0].data[...] = arg_params['out_fc1_weight'].asnumpy()
Net.params['out_fc1'][1].data[...] = arg_params['out_fc1_bias'].asnumpy()

In [11]:
Net.params['out_fc1'][0].data[...] - arg_params['out_fc1_weight'].asnumpy()
Net.params['out_fc1'][1].data[...] - arg_params['out_fc1_bias'].asnumpy()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [12]:
np.linalg.norm(Net.params['stem_conv1'][0].data[...])

0.38500732

In [16]:
Net.params

OrderedDict([('stem_conv1', <caffe._caffe.BlobVec at 0x7f9dfaaecd00>),
             ('stem_bn2', <caffe._caffe.BlobVec at 0x7f9e17769de0>),
             ('stem_bn2_scale', <caffe._caffe.BlobVec at 0x7f9dfab20520>),
             ('stem_relu1', <caffe._caffe.BlobVec at 0x7f9dfaaf5fa0>),
             ('stage1_unit1_bn1', <caffe._caffe.BlobVec at 0x7f9dfaaf5f30>),
             ('stage1_unit1_bn1_scale',
              <caffe._caffe.BlobVec at 0x7f9dfaaf8050>),
             ('stage1_unit1_conv1', <caffe._caffe.BlobVec at 0x7f9dfaaf80c0>),
             ('stage1_unit1_bn2', <caffe._caffe.BlobVec at 0x7f9dfaaf8130>),
             ('stage1_unit1_bn2_scale',
              <caffe._caffe.BlobVec at 0x7f9dfaaf81a0>),
             ('stage1_unit1_prelu2', <caffe._caffe.BlobVec at 0x7f9dfaaf8210>),
             ('stage1_unit1_conv2', <caffe._caffe.BlobVec at 0x7f9dfaaf8280>),
             ('stage1_unit1_bn4', <caffe._caffe.BlobVec at 0x7f9dfaaf82f0>),
             ('stage1_unit1_bn4_scale',
           

In [13]:
print(np.sum(Net.params['out_embedding_scale'][1].data[...]-arg_params['out_embedding_beta'].asnumpy()))
print(np.sum(Net.params['out_embedding_scale'][0].data[...]-arg_params['out_embedding_gamma'].asnumpy()))
print(np.sum(Net.params['out_embedding'][0].data[...]-aux_params['out_embedding_moving_mean'].asnumpy()))
print(np.sum(Net.params['out_embedding'][1].data[...]-aux_params['out_embedding_moving_var'].asnumpy()))

0.0
0.0
0.0
0.0


In [14]:
arg_params['out_embedding_gamma'].asnumpy()

array([0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999994,
       0.99999994, 0.99999994, 0.99999994, 0.99999994, 0.99999

In [15]:
Net.params['out_embedding_scale'][0].data[...]=1.0

In [16]:
out_caffe = getOutput('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0001.png', 'out_embedding')

In [17]:
np.dot(out_caffe/np.linalg.norm(out_caffe),out_mx/np.linalg.norm(out_mx))

1.0

In [18]:
Net.save('/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/resnet20_no_merge_no_flattern.caffemodel')

In [19]:
#cfModelProto='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.prototxt'
#cfModelWeight='/media/macul/black/mxnet_training/r50/dgx_train1/dgx_train1_31-caffe.caffemodel'
cfModelProto='/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/resnet20_no_merge_no_flattern.prototxt'
cfModelWeight='/home/macul/Documents/macul/FaceRecog_ResNet_MX/output/dgx_train7/resnet20_no_merge_no_flattern.caffemodel'
Net = caffe.Net(cfModelProto, cfModelWeight, caffe.TEST)
Net.blobs['data'].reshape(1,3,112,112)

In [20]:
out_caffe_new = getOutput('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0001.png', 'out_embedding')

In [21]:
np.dot(out_caffe_new/np.linalg.norm(out_caffe_new),out_mx/np.linalg.norm(out_mx))

1.0

In [22]:
# dgx_train1_31-caffe.prototxt and dgx_train1_31-caffe.caffemodel are converted caffe model for use
# note that the input channel should be RGB, image should be normalized, and the output embedding should be normalized by its norm

In [23]:
np.linalg.norm(out_caffe_new)

0.08831205

In [ ]:
# next to merge BN layer using merge-caffe-bn-conv-layer.ipynb